In [1]:
#Importing all the necessary Libraries
import pandas as pd
import re
# lxml easy-to-use library for processing XML and HTML
import lxml
from PIL import Image
from io import BytesIO
import os
import webbrowser
#Beautiful Soup is a Python package for parsing HTML and XML documents.
#Beautiful Soup is a library that makes it easy to scrape information from web pages.
# The requests.get() method sends a GET request to the specified URL
from bs4 import BeautifulSoup
from requests import get
%matplotlib inline

In [2]:
b=[]
# The Initial URL of the website we intend to Crawl is mentioned below
url= "https://www.imdb.com/list/ls068010962/"
b.append("https://www.imdb.com/list/ls068010962/")
#The get() method sends a GET request to the specified URL
page = get(url)
soup1 = BeautifulSoup(page.content, 'lxml')
c = soup1.find(id="main")
#list-pagination contains the information regarding the Number of Pages 
frame=c.find("div", class_="list-pagination")
Line=frame.find("span", class_="pagination-range")
x=list(Line)
y=x[0]
p=list(y.split(" "))
a=[]
for i in range(0,len(p)):
    if p[i].isdigit():
        a.append(p[i])
i=len(b)
#Here, list 'a' contains the details of the number of Actors Deatils present in the base URL
#and the total number of Celebrity details available. Here a=['1','100','200']
# With this list, we will be able to find out the total number of pages we need in order to get all the information.
if a[1]<a[2]:
    i+=1
    b.append(str(url)+"?page="+str(i))
    a[1]+=a[1]

In [3]:
#Create a class and define functions to get the data from the website
class IMDB(object):
    def __init__(self, url):
        super(IMDB, self).__init__()
            
    def manip(self,y):
        x="\n\n"
        c=0
        p=list(y.split(" "))
        for i in range(0,len(p)):
                if p[i]==x:
                    c+=1
                    if c==2:
                        return " ".join(str(s) for s in p[i+1:-1])
     # This is the function where data such as Actor Name, Most Recognized Movie, Description, 
    #Their Traits, Their Image,etc... are scraped and appended into their respective lists
    def Dataset_Generation(self,Frame,ActorName,ActorTopMovie,ActorDescription,ActorTraits,Sno,imagestr,s,g):
        for Actor,i in zip(Frame,range(s,g)):
            # Here Actor is the frame content.Actor.find() is used to find a particular class in the Frame
            Line1 = Actor.find("h3", class_="lister-item-header")
            # Once Class is identified, The Contents can be extrated as shown below.
            #the append() method is used to append the extracted contents to their respective lists.
            ActorName.append(Line1.find("a").text.replace("\n",""))
            Sno.append(re.sub(r"[()]","", Line1.find_all("span")[-1].text))
            Line2=Actor.find("p", class_="text-muted text-small")
            ActorTopMovie.append(Line2.find("a").text.replace("\n",""))
            #The text provided as <p> in HTML cannot be extracter directly. Hence, We use the get_text() method to acces the entire text.
            #the get_text() method returns a list of length 1.
            Line3=Actor.find_all("div", class_="lister-item-content")
            text = [e.get_text() for e in Line3]
            #this list content is saved in the variable y
            y=text[0]
            #with the text extracted, we find the Trait of the celebrity. 
            #Whether the person is an Actor, an Actress, Director, Writer or a Producer.
            if "Actor |" in y:
                ActorTraits.append("Actor")
            elif "Actress |" in y:
                ActorTraits.append("Actress")
            elif "Writer |" in y:
                ActorTraits.append("Writer")
            elif "Director |" in y:
                ActorTraits.append("Director")
            elif "Producer |" in y:
                ActorTraits.append("Producer") 
            else:
                ActorTraits.append("--NIL--")
            # To get the description from the text we call the already defined function self.manip()     
            ActorDescription.append(self.manip(y))
            Line4 = Actor.find("div", class_="lister-item-image")
            #In order to Scrap image , we need to get the content in the 'src' of the <img> Tag
            response = get(Line4.find('img').get("src"))
            #The Src file is opened and the image is converted to RGB
            bytes = BytesIO(response.content)
            image = Image.open(bytes).convert('RGB')
            # The converted imaged is saved into a folder "Images/"
            image.save("Images/" + str(i) +".jpg")
            # Now the saved image is appended into the imagestr list
            imagestr .append('<img src="{}" > '.format("Images/" + str(i) +".jpg"))
        
    # This is the main function where teh DataFrame is generated and .html file is created
    def actorData(self,):
        # All the required lists are created
        ActorName = []
        ActorTopMovie = []
        ActorDescription=[]
        ActorTraits=[]
        Sno = []
        imagestr=[]
        # A folder named 'Images/' is created to store the scraped Images
        if not os.path.exists("Images/"):
            os.mkdir("Images/")
        s=1
        g=101
        # For each url, i.e each page we need to scrap, the following process is done and the lists are appended
        # In this URL, there are a total of 200 Celebrities information and each page consists of 100 Celebrities data
        # Hence, we have to scrap two pages i.e len(b)=2 or there are 2 URL's to scrap
        for url in b:
            page = get(url)
            soup = BeautifulSoup(page.content, 'lxml')
            content = soup.find(id="main")
            Frame=content.find_all("div", class_="lister-item mode-detail")
            self.Dataset_Generation(Frame,ActorName,ActorTopMovie,ActorDescription,ActorTraits,Sno,imagestr,s,g)
            s+=100
            g+=100         
        #Creating the DataFrame using Pandas   
        ActorData = pd.DataFrame({'Sno': Sno,'Actor Name':ActorName})
        ActorData['Actor Image']=imagestr
        ActorData['Best Of Their Works']=ActorTopMovie
        ActorData['Actor Description']=ActorDescription
        ActorData['Trait']=ActorTraits
        #In order to include images into the DataFrame, The Path is required.
        def path_to_image_html(path):
            return '<img src="'+ path + '" width="60" >'
        # The pd.set_option() allows us to customize the DataFrame according to our requirements
        pd.set_option('display.max_colwidth', -1)
        pd.set_option('display.colheader_justify',"centre")
        ActorData.style.set_properties(subset=['Actor Image'], **{'width': '140px','height':'209'})
        #The HTML file is created . This .html is used to display the generated Database
        html=ActorData.to_html(escape=False ,formatters=dict(image=path_to_image_html))
        text_file = open(soup.find("h1", class_="header").text.replace("\n","")+".html", "w")
        text_file.write(html)
        text_file.close()
        #Opening the .html file in Chrome
        webbrowser.open('file:///C:/Users/LASYA/Top%20200%20Best%20Indian%20Actors%20and%20Actresses.html')

In [4]:
if __name__ == '__main__':
    # Creating the Class Object
    site1 = IMDB(url)
    site1.actorData()


C:\Users\LASYA\Anaconda3\lib\site-packages\PIL\Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
